#TP2 Ondelettes2D Debruitage Compression


# Transformée en Ondelettes 2D, application au traitement des images

In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import matplotlib.pyplot as plt
import pywt
import scipy.io as sio
import pandas as pd
import holoviews as hv
import param
import panel as pn
from panel.pane import LaTeX
hv.extension('bokeh')
from PIL import Image
from io import BytesIO
import requests

## Approximation linéaire et non linéaire.

In [ ]:
local=0
def chargeData(name):
    if local:
        if name=='Lenna':
            res=np.array(Image.open("./Archive/img/Lenna.jpg")).astype(float)
        if name=='Canaletto':
            res=np.array(Image.open("./Archive/img/Canaletto.jpeg")).astype(float)
        if name=='Minotaure':
            res=np.array(Image.open("./Archive/img/MinotaureBruite.jpeg")).astype(float)   
        if name=='Cartoon':
            res=np.array(Image.open("./Archive/img/Cartoon.jpg")).astype(float) 
    else:
        if name=='Lenna':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Lenna.jpg'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Canaletto':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Canaletto.jpeg'
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Minotaure':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/MinotaureBruite.jpeg'
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Cartoon':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Cartoon.jpg'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
    return res

In [ ]:
im2=chargeData('Lenna')
im=chargeData('Canaletto')
imagesRef= {"Lenna" : im2,"Canaletto" : im}
print("Les images de référence pour Lenna à gauche et de Canaletto à droite : ")
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
pn.Row(hv.Raster(imagesRef["Lenna"]).opts(**options),hv.Raster(imagesRef["Canaletto"]).opts(**options))

In [ ]:
size=400
WT= pywt.wavedecn(im, 'haar', mode='per', level=2)
arr, coeff_slices = pywt.coeffs_to_array(WT)
hv.Image(arr).opts(cmap='gray',width=size,height=size)

Ecrire une fonction qui réalise une approxiamtion non linéaire en seuillant les coefficients d'ondelettes.
On pourra utiliser les fonctions suivante : pywt.coeffs_to_array et pywt.array_to_coeffs

In [ ]:
def ApproxOnd2D(S,qmf,L,threshold):
    Lmax=pywt.dwt_max_level(len(S),pywt.Wavelet(qmf).dec_len)
    L1=min(L,Lmax)
    WT= pywt.wavedecn(S, qmf, mode='per', level=L1)
    arr, coeff_slices = pywt.coeffs_to_array(WT)
    WTS=arr*(np.abs(arr)>threshold)
    ncoeffs=(np.abs(arr)>threshold).sum()
    
    coeffs_from_arr = pywt.array_to_coeffs(WTS, coeff_slices)
    Srec=pywt.waverecn(coeffs_from_arr,qmf,mode='per')
    return Srec,ncoeffs

In [ ]:
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3','sym2','sym3']
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
im_res,ncoeffs=ApproxOnd2D(im,'db2',6,2)
hv.Raster(im_res).opts(**options)
print("Le nombre de coefficient pour réaliser l'approximation non linéaire est de", ncoeffs)

In [ ]:
class Approx2D(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(5,bounds=(0,7))
    T = param.Integer(7,bounds=(1,10))
    def view(self):
        im_res,ncoeffs = ApproxOnd2D(imagesRef[self.image],self.wave,self.L,self.T)
        options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
        return hv.Raster(im_res).opts(**options)

In [ ]:
approx2D = Approx2D()
pn.Row(approx2D.param,approx2D.view)

<font color='red'>
Nous effectuons une approximation linéaire des deux images (Lenna et Canaletto) pour différents types d'ondelettes en précisant le niveaux L et le seuil T. 

Visuellement, l'approximation linéaire nous donne un résultat proche des images initiales. Nous constatons que la qualité de l'image est meilleure lorsque $L$ augmente. Mais il est assez difficile de juger de visu la quanlité de l'approximation. Nous devons donc faire appel au PNSR (Peak Signal to Noise Ratio) qui mesure la qualité de reconstruction de l'image compressée par rapport à l'image initiale.

Il est défini par :
$$ PSNR = 20 * log_{10}(\frac{d}{EQM})$$
avec
    
- d : la valeur maximale pour un pixel.
    
- EQM  : l'erreur quadratique moyenne.
    
    </font>


Ecrire une fonction PSNR.

In [ ]:
def PSNR(I,Iref):
    
    mse = np.mean( (Iref - I) ** 2 )
    
    if mse == 0:
        return 100
    
    Val_MAX = np.max(I)
    return 20 * np.log10(Val_MAX / np.sqrt(mse))

Ecrire une fonction qui réalise une approximation non linéaire en conservant un nombre N de coefficients d'ondelettes et la tester. On pourra utiliser les fonctions pywt.ravel_coeffs et unravel_coeffs.

In [ ]:
def ApproxOnd2nonlin(S,qmf,L,N):
    a,b=np.shape(S)
    N1=a*b
    Lmax=pywt.dwt_max_level(len(S),pywt.Wavelet(qmf).dec_len)
    L1=min(L,Lmax)
    WT= pywt.wavedecn(S, qmf, mode='per', level=L1)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(WT)
    Ind=np.argsort(np.abs(arr))
    WTS=np.zeros(N1)
    WTS[Ind[N1-N:N1]]=arr[Ind[N1-N:N1]]
    coeffs_from_arr=pywt.unravel_coeffs(WTS,coeff_slices, coeff_shapes)
    
    Srec=pywt.waverecn(coeffs_from_arr,qmf,mode='per')
    p=PSNR(S,Srec)
    return Srec,p
    

In [ ]:
Irec,p=ApproxOnd2nonlin(im,'db2',6,5000)
hv.Image(Irec).opts(cmap='gray',width=400,height=400)
PSNR(im,Irec)

#print(p)

Créer un Dashboard qui permet d'explorer la fonction précédente.

In [ ]:
class Approx2Dnonlin(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(5,bounds=(0,7))
    N = param.Integer(2000,bounds=(1,10000))
  #  @param.depends('wave', 'N', 'L')
    def view(self):
        im_res,ncoeffs = ApproxOnd2nonlin(imagesRef[self.image],self.wave,self.L,self.N)
        psnr1=PSNR(imagesRef[self.image],im_res)
        strp1="%2.2f" % psnr1
        te1='PSNR signal reconstruit = '
        TN1=hv.Text(0.5,0.5,te1+strp1).opts(xaxis=None,yaxis=None,toolbar=None)
        options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
        return pn.Row(hv.Raster(im_res).opts(**options),TN1)

In [ ]:
approx2Dnonlin = Approx2Dnonlin()
pn.Row(approx2Dnonlin.param,approx2Dnonlin.view)

<font color='red'>
Tout d'abord, nous constatons que lorsqu'on augmente la valeur de N, la qualité d'image augmente. Quand N est environ 5000, l'image reconstruit n'est pas mal reconstruit. Il est en de même de pour les valeurs de $L$. En effet, à $L=0$, la qualité de l'image reconstruite est extrêment mauvaise.    
Concernant les bases, nous constatons globalement que pour les 2 images que la base $coif3$ semble donner donner le meilleur résultat. 
    </font>

Créer un plan d'experiences qui permet d'explorer la fonction ApproxOnd2nonlin

In [ ]:
import itertools
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']
experiences = {'Image':imagesRef,'N':np.linspace(1000,50000,30),'wave':wavelist}
dfexp = pd.DataFrame(list(itertools.product(*experiences.values())),columns=experiences.keys())

In [ ]:
print(dfexp)

Créer la fonction qui à une ligne de la base de donnée précédente calcule le PSNR associé.

In [ ]:

def row2PSNR(row):
    L=5
    I,psnr=ApproxOnd2nonlin(imagesRef[row.Image],row.wave,L,int(row.N))
    return {'PSNR':psnr}
    
 

Appliquer la fonction sur la base de donnée et ajouter la colonne PSNR à la base de données dfexp 

In [ ]:
result = dfexp.apply(row2PSNR,axis=1)
dfexp[['PSNR']] = pd.DataFrame.from_records(result.values)

In [ ]:
print(dfexp)

Utiliser hvplot pour visualiser la base de données.

In [ ]:
import hvplot.pandas


In [ ]:
from bokeh.models import HoverTool
h = HoverTool()

In [ ]:

dfexp.hvplot('N','PSNR',by='wave',kind='scatter',groupby=['Image'])\
.opts(width=600,tools = [h]).redim.range(PSNR=(10,125),N=(800,51000))


<font color='red'>
Ce graphe nous permet d'avoir un idée de l'effet de la valeur de $N$ et de la base de décomposition sur le PNSR et la reconstruction de l'image. 

On voit que pour les deux images, plus N augmente, plus PSNR augmente donc plus la qualité d'image reconstruit est bon. L'ondelette de "coif3" est la meilleure tandis que l'ondelette de "Haar" est la plus mauvaise. 
    </font>

## Débruitage d'images

Ecrire une fonction qui effectue un seuillage dur en ondelettes et la tester. On pourra utiliser la fonction pywt.ravel_coeffs et on pensera à cliper le résultat entre 0 et 255.

In [ ]:
def SeuillageDurOndelettes(I,qmf,L,Seuil):
    N1=np.shape(I)[0]*np.shape(I)[1]
    Lmax=pywt.dwt_max_level(np.shape(I)[0],pywt.Wavelet(qmf).dec_len)
    L1=min(L,Lmax)
    WT= pywt.wavedecn(I, qmf, mode='per', level=L1)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(WT)
    WTS=arr*(np.abs(arr)>Seuil)
    coeffs_from_arr = pywt.unravel_coeffs(WTS, coeff_slices, coeff_shapes)
    Irec=np.clip(pywt.waverecn(coeffs_from_arr,qmf,mode='per'),0,255)
    return Irec

Construire un dashboard qui permet d'explorer la fonction SeuillageDurOndelettes.

In [ ]:
class WaveSeuillage(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(7,bounds=(0,7))
    Seuil = param.Number(10,bounds=(1,100))
    def view(self):
        Iref=imagesRef[self.image]
        Irec=SeuillageDurOndelettes(imagesRef[self.image],self.wave,self.L,self.Seuil)
        options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
        return pn.Row(hv.Raster(Iref).opts(**options),hv.Raster(Irec).opts(**options))

In [ ]:
WaveSeuil=WaveSeuillage()
pn.Column(WaveSeuil.param,WaveSeuil.view)

<font color='red'>
Le seuillage en dur a pour effet de pixeliser l'image de départ. Cette fonction, permettant d'éliminer les petits variations du au bruit, nous permet de conserver que les coefficients nous permettant d'avoir le plus d'informations sur l'image initiale.
    </font>

Nous effectuons un test sur des images non bruités. Nous constatons que plus le seuil est grand, moins la quantité de l'image est bonne. 

In [ ]:
n1,n2=np.shape(im)
B=np.random.randn(n1,n2)
sigma=10
ib=im+sigma*B
ib=np.clip(ib,0,255)
hv.Image(ib).opts(cmap='gray',width=400,height=400)


Ecrire un dashboard qui permet de visualiser rapidement l'effet d'un débruitage en ondelettes et qui renvoie les images originales, bruitées et débruitées ainsi que les PSNR associés aux images bruitéeset débruitées.

In [ ]:
class WaveDebruit(param.Parameterized):
    image = param.ObjectSelector(default="Lenna",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(7,bounds=(0,7))
    Seuil = param.Number(3,bounds=(1,6))
    Sigma = param.Number(10,bounds=(1,30))
    seednoise = param.Integer(1,bounds=(0,50))
    def view(self):
        Iref=imagesRef[self.image]
        n1,n2=np.shape(Iref)
        B=np.random.randn(n1,n2)
        ib=Iref+self.Sigma*B
        ib=np.clip(ib,0,255)
        Irec=SeuillageDurOndelettes(ib,self.wave,self.L,sigma*self.Seuil)
        psnr1=PSNR(imagesRef[self.image],Irec)
        strp1="%2.2f" % psnr1
        strp2="%2.2f" % PSNR(ib,Iref)
        te1='PSNR Image reconstruit = '
        te2='PSNR Image bruité = '
        TN1=hv.Text(0.5,0.7,te1+strp1).opts(xaxis=None,yaxis=None,toolbar=None)
        TN2=hv.Text(0.5,0.5,te2+strp2).opts(xaxis=None,yaxis=None,toolbar=None)
        options = dict(cmap='gray',xaxis=None,yaxis=None,width=350,height=350,toolbar=None)
        return pn.Row(pn.Column(hv.Raster(Iref).opts(**options),hv.Raster(ib).opts(**options),hv.Raster(Irec).opts(**options)),TN1*TN2)#,PSNR(Iref,ib),PSNR(Iref,Irec)


In [ ]:
WaveDeb=WaveDebruit()
pn.Row(WaveDeb.param,WaveDeb.view)

<font color='red'>
Pour cette expérience, nous avons 3 différentes images: la première est l'image originale, la deuxième l'image bruitée et la dernière l'image reconstruite. Nous allons observer les effets des différents paramètres sur le résultat du programme. Nous allons observer ces effets avec la base d'ondelettes $coif3$. 

En observant, les variations de PSNR suivant les valeurs du paramèttre $seuil$, nous constatons que le choix optimal est la valeur $3$, pour $sigma=10$. En effet, pour $seuil=3$, nous obtenons la plus grande valeur du PSNR. Néamoins, lorsque sigma varie, cette valeur change. Toutefois, la combinaision optimale de paramètres reste $seuil=3$ et $sigma=10$.
</font>


## Débruitage d'images et translations

Ecrire une fonction qui réalise un débruitage avec une moyenne sur des NbT fois NbT translations et la tester. Vérifier le gain en PNSR.

In [ ]:
def DebruitTranslation(IB,I,wave,seuil,NbT):
    n1,n2=np.shape(IB)
    Lmax=pywt.dwt_max_level(n1,pywt.Wavelet(wave).dec_len)
    Isum=0*IB
    P=np.zeros(NbT*NbT)
    for i in range(0,NbT):
        for j in range(0,NbT):
            Ibt=np.roll(IB,(i,j))
            Irectemp=SeuillageDurOndelettes(Ibt,wave,Lmax,seuil)
            Irectemp2=np.roll(Irectemp,(-i,-j))
            Isum=Isum+Irectemp2
            Irec=Isum/(NbT*i+j+1)
            P[NbT*i+j]=PSNR(Irec,I)
    return Irec,P        

In [ ]:
n1,n2=np.shape(im)
B=np.random.randn(n1,n2)
sigma=10
ib=im+sigma*B
ib=np.clip(ib,0,255)
ir,PnSr=DebruitTranslation(ib,im,"haar",7,5)
print("PNSR = ",PnSr)
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
pn.Row(hv.Raster(ib).opts(**options),hv.Raster(ir).opts(**options))


Créer un dasboard pour explorer la fonction précédente. La sortie doit aussi être composée de 3 images et 2 PSNR.

In [ ]:
class Debruit_translat(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    NbT = param.Integer(2,bounds=(1,8))
    Sigma = param.Number(10,bounds=(1,30))
    seednoise = param.Integer(1,bounds=(0,50))
    def view(self):
        seuil=3*self.Sigma
        Iref=imagesRef[self.image]
        n1,n2=np.shape(Iref)
        np.random.seed(seed=self.seednoise)
        B=np.random.randn(n1,n2)
        ib=Iref+self.Sigma*B
        ib=np.clip(ib,0,255)
        Irec,Pns=DebruitTranslation(ib,Iref,self.wave,seuil,self.NbT)
        options = dict(cmap='gray',xaxis=None,yaxis=None,width=350,height=350,toolbar=None)
        strp1="%2.2f" % Pns[-1]
        strp2="%2.2f" % PSNR(ib,Iref)
        te1='PSNR Image reconstruit = '
        te2='PSNR Image bruité = '
        TN1=hv.Text(0.5,0.7,te1+strp1).opts(xaxis=None,yaxis=None,toolbar=None)
        TN2=hv.Text(0.5,0.5,te2+strp2).opts(xaxis=None,yaxis=None,toolbar=None)
        return pn.Row(pn.Column(hv.Raster(Iref).opts(**options),hv.Raster(ib).opts(**options),hv.Raster(Irec).opts(**options)),TN2*TN1)

In [ ]:
Deb_trans=Debruit_translat()
pn.Row(Deb_trans.param,Deb_trans.view)

<font color="red">
Pour cette expérience, nous avons 3 différentes images: la première est l'image originale, la deuxième l'image bruitée et la dernière l'image reconstruite. Nous constatons tout d'abord que le résultat obtenu est meilleur que pour la méthode précédente avec les mêmes paramètres en observant la valeur du PSNR. Nous constatons que le nombre optimal de translations à effectuer fortement de la valeur de $sigma$. Néanmoins, nous notons que la valeur du PNSR décroit à partir de $NbT \geq 1$, puis croit.
Avec cette méthode, à chaque translation, un débruitage d'une image est faite. Ainsi en faisant cela, nous obtenons un moyenne permettant d'aténuer le bruitage et de se rapprocher de l'image initiale.
    </font>

## Débruitage d'une image couleur.

In [ ]:
im=chargeData('Cartoon').astype('uint8')

im2=chargeData('Minotaure').astype('uint8')

imagesRef_col= {"Minotaure" : im2,"Cartoon" : im}
options1=dict(width=400,height=400,xaxis=None,yaxis=None,toolbar=None)
hv.RGB(im).opts(**options1)

Pour effectuer le débruitage d'une image générale, c'est à dire d'une image couleur dont le format n'est pas carré et dont les dimensions ne sont pas des puissnaces de 2 on procède comme suit :

1) On effectue un débruitage séparé sur chacun des canaux.

2) Le format carré n'est pas un vraiu problème, il faut juste que les dimensions soit des multiples de puissances de 
2. C'est la puissance de 2 qui définira l'échalle maximale de la décomposition en ondelettes. Il est donc préférable que les dimensions de l'images soient un petit multiple d'une puissance de 2.

3) On étend l'image par symétrie ou périodicité pour qu'elle ait les dimensions souhaitées. A la fin du processus de débruitage on tronque le résultat obtenu à la dimension de l'image originale.

4) Si le niveau de bruit n'est pas connu, il faut l'estimer en utilisant les coefficients d'ondelettes de la plus petite échelle (voir le notebook sur le débruitage de signaux).


In [ ]:
def PSNRColor(im,iref):
    mse = 0
    Val_MAX = 0
    for i in range (3):
        mse += sum((np.ravel(iref[:,:,i] - im[:,:,i])) ** 2)
        Val_MAX = max(Val_MAX, max(np.ravel(iref)))
    mse = mse / 3 / iref.size
    if mse == 0:
        return 100
    return 20 * np.log10(Val_MAX / np.sqrt(mse))

In [ ]:
def periodique(I):
    n1,n2 = np.shape(I)
    im = np.zeros((2*n1,2*n2))
    im[:n1,:n2] = I
    im[n1:,:n2] = I
    im[:n1,n2:] = I
    im[n1:,n2:] = I
    return im
    
def symetrique(I):
    n1,n2 = np.shape(I)
    im = np.zeros((2*n1,2*n2))
    im[:n1,:n2] = I
    im[n1:,:n2] = np.flipud(I)
    im[:n1,n2:] = np.fliplr(I)
    im[n1:,n2:] = I
    return im   

def EstimEcartTypeBruit(I,qmf):
    n = np.shape(I)
    Lmax=pywt.dwt_max_level(min(n[0],n[1]),pywt.Wavelet(qmf).dec_len)
    wsb=pywt.wavedec(I, qmf, mode='per', level=Lmax)
    mt=np.sqrt(2)*scp.special.erfinv(0.5)
    return np.median(np.abs(wsb[Lmax]))/mt

In [ ]:
def debruit_color(I,qmf,L,seuil):
    I_debrui=np.copy(I)
    n1,n2,n3=np.shape(I)
    I_debrui[:,:,0]=SeuillageDurOndelettes(I[:,:,0],qmf,L,seuil)
    I_debrui[:,:,1]=SeuillageDurOndelettes(I[:,:,1],qmf,L,seuil)
    I_debrui[:,:,2]=SeuillageDurOndelettes(I[:,:,2],qmf,L,seuil)
    return I_debrui

In [ ]:
def debruit(I,qmf,L,seuil):
    I_debrui=np.copy(I)
    I_debrui=SeuillageDurOndelettes(I,qmf,L,seuil)    
    return I_debrui

def debruit_color3(IB,qmf,per,L,seuil,sigma=None,):
    n = np.shape(IB)
    isImage = len(n) == 3
    im = np.zeros(n)
    #seuil = 3
    if isImage :
        #si image est colorée
        for i in range(isImage*2+1):
            #pour obtenir une image dont dimensions sont un multiple de 2
            if per == 1 :
                I = periodique(IB[:,:,i])
            else :
                I = symetrique(IB[:,:,i])
            #pour evaluer le niveau de bruit
            if sigma == None : 
                sigma = float(EstimEcartTypeBruit(IB[:,:,i],qmf))
            #debruite en prenant un seuil de seuil* niveau de bruit
            im_aux = debruit(I,qmf,L,seuil*sigma)
            #pour retrovuer image de départ débuitee
            im[:,:,i] = im_aux[:n[0],:n[1]]
    else :
        #si image est en niveau de gris
        #estime le niveau de bruit
        if sigma== None : 
            sigma = float(EstimEcartTypeBruit(IB,qmf))
        #debruite en prenant un seuil de 3* niveau de bruit
        im = debruit(IB,qmf,L,seuil*sigma)
        #pour retrouver image de depart debruitee
        im = im[:n[0],:n[1]]             
    return im

In [ ]:
n1,n2,n3=np.shape(im)
bruitcouleur=np.random.randn(n1,n2,3)
sigma=10
imb=im+sigma*bruitcouleur
im_re=debruit_color(imb,'db2',7,3*10)
PSNR(im,im_re)
hv.RGB(im_re.astype('uint8')).opts(**options1)

# Proposer une fonction qui effectue le débruitage d'une image couleur de dimensions quelconques. 

In [ ]:
class WaveDebruit_col(param.Parameterized):
    image = param.ObjectSelector(default="Cartoon",objects=imagesRef_col.keys())
    wave = param.ObjectSelector(default="db2",objects=wavelist)
    L = param.Integer(7,bounds=(0,7))
    Seuil = param.Number(3,bounds=(1,6))
    Sigma = param.Number(10,bounds=(1,30))
    seednoise = param.Integer(1,bounds=(0,50))
    per=param.Integer(1,bounds=(1,2))
    def view(self):
        Iref=imagesRef_col[self.image]
        n1,n2,n3=np.shape(Iref)
        np.random.seed(seed=self.seednoise)
        bruitcouleur=np.random.randn(n1,n2,3)
        imb=Iref+self.Sigma*bruitcouleur
        imb=np.clip(imb,0,255)
        per=1
        Irec=debruit_color3(imb,self.wave,self.per,self.L,self.Seuil,self.Sigma)
        psnr1=PSNRColor(imagesRef_col[self.image],Irec)
        strp1="%2.2f" % psnr1
        strp2="%2.2f" % PSNRColor(imb,Iref)
        te1='PSNR Image reconstruit = '
        te2='PSNR Image bruité = '
        TN1=hv.Text(0.5,0.7,te1+strp1).opts(xaxis=None,yaxis=None,toolbar=None)
        TN2=hv.Text(0.5,0.5,te2+strp2).opts(xaxis=None,yaxis=None,toolbar=None)
        options=dict(width=350,height=350,xaxis=None,yaxis=None,toolbar=None)
        return pn.Row(pn.Column(hv.RGB(Iref.astype('uint8')).opts(**options),hv.RGB(imb.astype('uint8')).opts(**options),hv.RGB(Irec.astype('uint8')).opts(**options)),TN2*TN1)


In [ ]:
WaveDeb_col=WaveDebruit_col()
pn.Row(WaveDeb_col.param,WaveDeb_col.view)

<font color="red">
Pour cette expérience, nous avons 3 différentes images: la première est l'image originale, la deuxième l'image bruitée et la dernière l'image reconstruite. Globalement, nous notons une amélioration du PSNR et donc nous avons réussi à diminuer les effets du bruit. 

Nous constatons aussi que la méthode de prolongement par symétrie donne un résultat légèrement meilleur que la méthode par périodicité. Néanmoins les résultats restent très proche.


</font>





La fonction peut prendre en entrée un tableau numpy ou une image dans une format d'images classique.
Vous pouvez tester votre programme en bruitant vous même une ou plusieurs images de référence et évaluer le gain en terme de PSNR.

## Pour aller plus loin (à titre informatif et optionnel)

On peut améliorer les méthodes par seuillage dans une base d'ondelettes en effectuant un seuillage par blocs. C'est à dire, ne pas décider de conserver ou pas un coefficients en fonction de sa seule amplitude mais plutôt en fonction de l'énergie d'un voisinage de coefficients. 

Voir : http://www.cnrs.fr/insmi/spip.php?article265

En effet, il est rare qu'un coefficient soit significatif seul au milieu de coefficients nuls. 

La mméthode de sueillage par blocs consiste à choisir une taille de voisinage (par exemple 4*4 coeffients en dimension 2) pour une échelle et une direction donnée et de conserver l'intégralité des coefficients si l'énergie (la somme des carrés des coefficients) est supérieure à un seuil et de les mettre tous à 0 si ce n'est pas le cas. 

Dans ce cas aussi, les translations permettent d'améliorer le rendu visuel en limitant les effets de blocs.

On peut aussi constuire des blocs "3D" en considérant des blocs qui comprennent les coefficients des 3 créneaux de couleurs. L'idée est de corréler le débruitage un peu à travers l'espace et l'espace des couleurs.

Il est possible d'effectuer un débruitage en changeant d'espace colorimétrique en passant du RGB au YUV par exemple.

# Débruiter un minotaure ?

A l'aide de tout ce qui a été fait précédemment, proposer une version débruitée de l'image couleur contenue dans le tableau Mi

In [ ]:
Mi=chargeData('Minotaure')
Minotaure=np.clip(Mi,0,255)
hv.RGB(Minotaure.astype('uint8')).opts(xlabel=None,ylabel=None,width=400,height=500)

In [ ]:
Irec = debruit_color3(Minotaure,'db2',1,10,3)

pn.Row(hv.RGB(Minotaure.astype('uint8')).opts(xlabel=None,ylabel=None,width=400,height=500),
       hv.RGB(Irec.astype('uint8')).opts(xlabel=None,ylabel=None,width=400,height=500))

In [ ]:
print("PSNR image débruité : ",PSNRColor(Irec,Minotaure))

Rédiger également une fonction prenant en entrée un nom de fichier 
permettant de calculer le PSNR de votre proposition d'image débruitée avec l'image en question.
On calcule le PSNR entre deux images couleurs en calculant la somme des erreurs quadratiques sur les 3 canaux.

Attention, l'image a 3 canaux de couleur, n'est pas carrée et les dimensions ne sont pas des puissances de 2.

### Plan d'expériences pour évaluer l'impact des translations  

Créer un plan d'expériences pour explorer les performances de l'invariance par translation pour le débruitage. 

In [1]:
print(imagesRef.keys())

NameError: ignored

In [ ]:
experiences_DebruitTrans = {'Image':imagesRef.keys(),'NbT':np.arange(1,5),'Sigma':np.linspace(10,30,2),'wave':wavelist}
dfexp_DebruitTrans = pd.DataFrame(list(itertools.product(*experiences_DebruitTrans.values())),columns=experiences_DebruitTrans.keys())

In [ ]:
print(dfexp_DebruitTrans)

Ecrire une fonction qui calcule le PSNR moyen sur n réalisations de bruit du débruitage d'une image avec NbT*NbT translations (qui utilise par exemple la fonction DebruitTranslation)

In [ ]:
def DebruitTranslation2(IB,wave,seednoise,sigma,NbT):
    n1,n2=np.shape(IB)
    Lmax=pywt.dwt_max_level(n1,pywt.Wavelet(wave).dec_len)
    Isum=0*IB
    np.random.seed(seed=seednoise)
    bruit=np.random.randn(n1,n2)
    P=np.zeros(NbT*NbT)
    seuil=3*sigma
    IBb=IB+sigma*bruit
    for i in range(0,NbT):
        for j in range(0,NbT):
            Ibt=np.roll(IBb,(i,j))
            Irectemp=SeuillageDurOndelettes(Ibt,wave,Lmax,seuil)
            Irectemp2=np.roll(Irectemp,(-i,-j))
            Isum=Isum+Irectemp2
            Irec=Isum/(NbT*i+j+1)
            P[NbT*i+j]=PSNR(IB,Irec)
    return Irec,P

def Debruit_Translat_PSNRMoyen(I,wave,sigma,NbT,n):
    P=np.zeros(NbT*NbT)
    for seednoise in np.arange(0,n):
        Irec,Ptemp=DebruitTranslation2(I,wave,seednoise,sigma,NbT)
        P=P+Ptemp
    P=P/n
    return P

Ecrire la fonction qui à une ligne de la base de données précédente calcule le PSNR moyen sur 4 réalisations du bruit. Puis l'appliquer à la base de données et ajouter la colonne des PSNR calculés à la base de données.

In [ ]:
def row2DebruitTrans(row):
    psnr=Debruit_Translat_PSNRMoyen(imagesRef[row.Image],row.wave,row.Sigma,row.NbT,4)
    return {'PSNR':psnr}

In [ ]:
resultTrans = dfexp_DebruitTrans.apply(row2DebruitTrans,axis=1)
dfexp_DebruitTrans[['PSNR']] = pd.DataFrame.from_records(resultTrans.values)

In [ ]:
print(dfexp_DebruitTrans)

In [ ]:
df = dfexp_DebruitTrans.copy()

df = pd.concat((df[['Image','NbT','wave','Sigma']],pd.DataFrame(df.PSNR.values.tolist(),df.index)),axis=1)

df = df.melt(id_vars=['Image','NbT','wave','Sigma'],var_name='translation',value_name='PSNR')

In [ ]:
print(df)

Utiliser hvplot pour visualiser les résulatst contenus dans la base de données.

In [ ]:
import hvplot.pandas
from bokeh.models import HoverTool
h = HoverTool()
df.hvplot('NbT','PSNR',by='wave',kind='scatter',groupby=['Image','Sigma'])\
.opts(width=600,tools = [h]).redim.range(PSNR=(20,35),NbT=(-0.5,5))

<font color="red">
Nous constatons que plus $NbT$ augmente, plus est le PSNR est grand. Cela signifie que la même utilisée obtient de plus en plus de coefficients permettant de réduire l'effet du bruit.  
    
</font>    

# Quantification et Entropie de Shannon

In [ ]:
def ShannonEntropy(x):
    value,counts = np.unique(x, return_counts=True)
    Proba=counts/len(x)
    Ent=-np.sum(np.log2(Proba)*Proba)
    return Ent

In [ ]:
x=np.array([13,13,2,7,13,7,1,13])
print(ShannonEntropy(x))

In [ ]:
y=np.array([-2,-3,1,0,1,0,-2,-3])
print(ShannonEntropy(y))

Ecrire une fonction qui effectue la quantification de la transformée en ondelettes avec un pas "Pas". On pourra à nouveau utiliser la commande pywt.ravel_coeffs. La fonction doit renvoyer l'image calculée par quantification, le PSNR associé ainsi que le nombre d'octets estimé par la valeur de l'entropie a priori nécessaire pour coder une telle image. On considérera qu'on code séparément les coefficients d'échelle et les coefficients d'ondelettes. Tester la fonction.

Créer le dashboard asscoié à la focntion précédente. 
Le dashboard doit renvoyer l'image quantifiée, le PSNR de l'image ainsi que le facteur de compression théorique associé. 

In [ ]:
class WaveQuant(param.Parameterized):
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    QS = param.Number(30,bounds=(10,300))
    ...

Créer dun plan d'expériences pour comparer les différentes ondelettes pour la quantification... et poursuivre jusqu'à obtenir un affichage de la base de données ainsi créée avec hvplot.

## Pour aller plus loin (à titre informatif et optionnel)

Nous proposons ici d'effectuer la compression sur les 3 canau RGB. Or l'oeil humain est plus sensible à la luminance qu'aux composantes purement chromatiques. C'est pourquoi, la plupart des algorithmes de compressions sont effectué dans un espace colorimétrique YUV où Y est la luminance. On alloue alors plus d'information au canal Y et on comprime plus drastiquement les deux autres canaux. Une méthode standart consiste par exemple à sous-échantionner d'un facteur 2 les deux composantes U et V avant de les comprimer. 

https://fr.wikipedia.org/wiki/Sous-échantillonnage_de_la_chrominance

On obtient alors des images de chrominances moins résolues et donc moins lourdes mais le rendu final reste correct car l'oeil humain est nettement plus sensible à la luminance. 